### Database Setup

Goals:

1. Provision PostgreSQL database for processing of SWC temperature data
2. Load NCEI, PRISM, NLDAS and GLDAS temperature datasets to PostgreSQL

Notes:

* The following code will work on any modern version of PostgreSQL.  More recent versions are better but not a requirement.
* At this time the PostGIS GIS processing extension is not required.
* The SWC temperature data, particularly NLDAS is challenging in its size (1,198,868,348 rows) meaning the more memory and processor power the better.  
* At least 75 GB of staging space and 75 GB of database space is required.  It may be possible to get by with 2/3 of that if the files and tables are deleted after each step.  However doing so makes QA and reprocessing impossible.
* Utilizing multiple tablespaces on multiple disks will provide the usual boost to performance.  Alter the SQL as needed to spread out tables and indexes on different tablespaces if possible.


#### As the database superuser
```
CREATE USER swc WITH PASSWORD 'swc123';

CREATE DATABASE swcdb OWNER swc;
```

#### As the swc user in the database swcdb
```
CREATE SCHEMA swc IF NOT EXISTS AUTHORIZATION swc;

CREATE TABLE swc.ncei_stats(
     stationid      VARCHAR(255) NOT NULL
    ,record_count   INTEGER      NOT NULL
);

CREATE TABLE swc.ncei_raw(
     stationid      VARCHAR(255) NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,tmax           NUMERIC      NOT NULL
    ,tmin           NUMERIC      NOT NULL
    ,station_source VARCHAR(255) NOT NULL
);

CREATE TABLE swc.prism_raw(
     stationid      VARCHAR(255) NOT NULL
    ,lon            NUMERIC      NOT NULL
    ,lat            NUMERIC      NOT NULL
    ,elev           NUMERIC      NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,tmin           NUMERIC      NOT NULL
    ,tmax           NUMERIC      NOT NULL
);

CREATE TABLE swc.nldas_raw(
     stationid      VARCHAR(255) NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,tempf          NUMERIC      NOT NULL
);

CREATE TABLE swc.gldas_raw(
     stationid      VARCHAR(255) NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,tempf_3hrlkbck NUMERIC      NOT NULL
);

CREATE TABLE swc.gldasnear_raw(
     stationid      VARCHAR(255) NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,tempf_3hrlkbck NUMERIC      NOT NULL
    ,near_dist      NUMERIC      NOT NULL
    ,near_gridx     NUMERIC      NOT NULL
    ,near_gridy     NUMERIC      NOT NULL
);

CREATE TABLE swc.station_stats(
     objectid          INTEGER      NOT NULL
    ,stationid         VARCHAR(255) NOT NULL
    ,has_some_ncei     VARCHAR(1)
    ,has_complete_ncei VARCHAR(1)
    ,has_prism         VARCHAR(1)
    ,has_nldas         VARCHAR(1)
    ,has_gldas         VARCHAR(1)
    ,has_gldasnear     VARCHAR(1)
    ,source_used       VARCHAR(255)
);

CREATE SEQUENCE swc.station_stats_seq START WITH 1;

CREATE TABLE swc.station_daily_temperatures(
     objectid       INTEGER      NOT NULL
    ,stationid      VARCHAR(255) NOT NULL
    ,station_date   TIMESTAMP WITHOUT TIME ZONE NOT NULL
    ,ncei_tmin      NUMERIC
    ,ncei_tmax      NUMERIC
    ,prism_tmin     NUMERIC
    ,prism_tmax     NUMERIC
    ,nldas_tmin     NUMERIC
    ,nldas_tmax     NUMERIC
    ,gldas_tmin     NUMERIC
    ,gldas_tmax     NUMERIC
    ,gldasnear_tmin NUMERIC
    ,gldasnear_tmax NUMERIC
);

CREATE SEQUENCE swc.station_daily_temperatures_seq START WITH 1;
```

#### As the swc user in the database swcdb with file access to the raw SWC CSV files
```
COPY swc.ncei_stats    FROM '/myfilelocation/ncei_20210805_stats.csv' DELIMTER ',' CSV;

COPY swc.ncei_raw      FROM '/myfilelocation/ncei_20210805.csv'       DELIMTER ',' CSV;

COPY swc.prism_raw     FROM '/myfilelocation/prism_20210816.csv'      DELIMITER ',' CSV HEADER;

COPY swc.nldas_raw     FROM '/myfilelocation/nldas_20210818.csv'      DELIMITER ',' CSV;

COPY swc.gldas_raw     FROM '/myfilelocation/gldas_20210822.csv'      DELIMITER ',' CSV;

COPY swc.gldasnear_raw FROM '/myfilelocation/gldasnear_20210822.csv'  DELIMITER ',' CSV;
```

#### As the swc user in the database swcdb
```
ALTER TABLE swc.ncei_stats
ADD CONSTRAINT ncei_raw_pk PRIMARY KEY(stationid);

VACUUM FULL ANALYZE swc.ncei_stats;

ALTER TABLE swc.ncei_raw
ADD CONSTRAINT ncei_raw_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.ncei_raw;

ALTER TABLE swc.prism_raw
ADD CONSTRAINT prism_raw_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.prism_raw;

ALTER TABLE swc.nldas_raw
ADD CONSTRAINT nldas_raw_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.nldas_raw;

ALTER TABLE swc.gldas_raw
ADD CONSTRAINT gldas_raw_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.gldas_raw;

ALTER TABLE swc.gldasnear_raw
ADD CONSTRAINT gldasnear_raw_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.gldasnear_raw;
```

### As the swc user in the database swcdb
```
INSERT INTO swc.station_stats(
     objectid 
    ,stationid
) 
SELECT
 nextval('swc.station_stats_seq')
,a.stationid
FROM
swc.ncei_stats a;

ALTER TABLE swc.station_stats
ADD CONSTRAINT station_stats_pk PRIMARY KEY(stationid);

ALTER TABLE swc.station_stats
ADD CONSTRAINT stations_stats_u01 UNIQUE(objectid);

CREATE INDEX station_stats_u02
ON swc.station_stats USING btree(has_some_ncei);

CREATE INDEX station_stats_u03
ON swc.station_stats USING btree(has_complete_ncei);

CREATE INDEX station_stats_u04
ON swc.station_stats USING btree(has_prism);

CREATE INDEX station_stats_u05
ON swc.station_stats USING btree(has_nldas);

CREATE INDEX station_stats_u06
ON swc.station_stats USING btree(has_gldas);

CREATE INDEX station_stats_u07
ON swc.station_stats USING btree(has_gldasnear);

INSERT INTO swc.station_daily_temperatures(
     objectid
    ,stationid
    ,station_date
)
SELECT
 nextval('swc.station_daily_temperatures_seq')
,a.stationid
,b.station_date
FROM
swc.station_stats a
CROSS JOIN (
    SELECT 
    i::TIMESTAMP AS station_date 
    FROM 
    generate_series('1990-01-01','2020-12-31','1 day'::interval) i
) b;

ALTER TABLE swc.station_daily_temperatures
ADD CONSTRAINT station_daily_temperatures_pk PRIMARY KEY(stationid, station_date);

ALTER TABLE swc.station_daily_temperatures
ADD CONSTRAINT station_daily_temperatures_u01 UNIQUE(objectid);

CREATE INDEX station_daily_temperatures_u02
ON swc.station_daily_temperatures USING btree(ncei_tmin);

CREATE INDEX station_daily_temperatures_u03
ON swc.station_daily_temperatures USING btree(ncei_tmax);

CREATE INDEX station_daily_temperatures_u04
ON swc.station_daily_temperatures USING btree(prism_tmin);

CREATE INDEX station_daily_temperatures_u05
ON swc.station_daily_temperatures USING btree(prism_tmax);

CREATE INDEX station_daily_temperatures_u06
ON swc.station_daily_temperatures USING btree(nldas_tmin);

CREATE INDEX station_daily_temperatures_u07
ON swc.station_daily_temperatures USING btree(nldas_tmax);

CREATE INDEX station_daily_temperatures_u08
ON swc.station_daily_temperatures USING btree(gldas_tmin);

CREATE INDEX station_daily_temperatures_u09
ON swc.station_daily_temperatures USING btree(gldas_tmax);

CREATE INDEX station_daily_temperatures_u10
ON swc.station_daily_temperatures USING btree(gldasnear_tmin);

CREATE INDEX station_daily_temperatures_u11
ON swc.station_daily_temperatures USING btree(gldasnear_tmax);

VACUUM FULL ANALYZE swc.station_daily_temperatures;
```   

### As the swc user in the database swcdb
```
UPDATE swc.station_daily_temperatures a
SET (ncei_tmin,ncei_tmax) = (
   SELECT
    b.tmin::NUMERIC
   ,b.tmax::NUMERIC
    FROM
    swc.ncei_raw b 
    WHERE
        b.stationid    = a.stationid
    AND b.station_date = a.station_date
);

UPDATE swc.station_daily_temperatures a
SET (prism_tmin,prism_tmax) = (
   SELECT
    b.tmin
   ,b.tmax
    FROM
    swc.prism_raw b 
    WHERE
        b.stationid    = a.stationid
    AND b.station_date = a.station_date
);

CREATE TABLE swc.nldas_day_agg 
AS
SELECT
 a.stationid
,DATE_TRUNC('day',a.station_date) AS station_date
,MIN(a.tempf) AS tmin
,MAX(a.tempf) AS tmax
FROM
swc.nldas_raw a
GROUP BY
 a.stationid
,DATE_TRUNC('day',a.station_date);

ALTER TABLE swc.nldas_day_agg
ADD CONSTRAINT nldas_day_agg_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.nldas_day_agg;

UPDATE swc.station_daily_temperatures a
SET (nldas_tmin,nldas_tmax) = (
   SELECT
    b.tmin
   ,b.tmax
    FROM
    swc.nldas_day_agg b 
    WHERE
        b.stationid    = a.stationid
    AND b.station_date = a.station_date
);


CREATE TABLE swc.gldas_day_agg 
AS
SELECT
 a.stationid
,DATE_TRUNC('day',station_date_3hourlookback - interval '3 hour') as station_date
,MIN(a.tempf) AS tmin
,MAX(a.tempf) AS tmax
FROM
swc.gldas_raw a
GROUP BY
 a.stationid
,DATE_TRUNC('day',station_date_3hourlookback - interval '3 hour');

ALTER TABLE swc.gldas_day_agg
ADD CONSTRAINT gldas_day_agg_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.gldas_day_agg;

UPDATE swc.station_daily_temperatures a
SET (gldas_tmin,gldas_tmax) = (
   SELECT
    b.tmin
   ,b.tmax
    FROM
    swc.gldas_day_agg b 
    WHERE
        b.stationid    = a.stationid
    AND b.station_date = a.station_date
);

CREATE TABLE swc.gldasnear_day_agg 
AS
SELECT
 a.stationid
,DATE_TRUNC('day',station_date_3hourlookback - interval '3 hour') as station_date
,MIN(a.tempf) AS tmin
,MAX(a.tempf) AS tmax
FROM
swc.gldasnear_raw a
GROUP BY
 a.stationid
,DATE_TRUNC('day',station_date_3hourlookback - interval '3 hour');

ALTER TABLE swc.gldasnear_day_agg
ADD CONSTRAINT gldasnear_day_agg_pk PRIMARY KEY(stationid,station_date);

VACUUM FULL ANALYZE swc.gldas_day_agg;

UPDATE swc.station_daily_temperatures a
SET (gldasnear_tmin,gldasnear_tmax) = (
   SELECT
    b.tmin
   ,b.tmax
    FROM
    swc.gldasnear_day_agg b 
    WHERE
        b.stationid    = a.stationid
    AND b.station_date = a.station_date
);

VACUUM FULL ANALYZE swc.station_daily_temperatures;
```

#### As the swc user in the database swcdb
```
UPDATE swc.station_stats a
SET 
has_some_ncei = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.ncei_tmin IS NOT NULL
) > 0;

UPDATE swc.station_stats a
SET 
has_complete_ncei = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.ncei_tmin IS NOT NULL
) = ( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures c 
    WHERE 
        c.stationid = a.stationid 
);


UPDATE swc.station_stats a
SET 
has_prism = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.prism_tmin IS NOT NULL
) > 0;

UPDATE swc.station_stats a
SET 
has_nldas = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.nldas_tmin IS NOT NULL
) > 0;

UPDATE swc.station_stats a
SET 
has_gldas = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.gldas_tmin IS NOT NULL
) > 0;

UPDATE swc.station_stats a
SET 
has_gldasnear = 'Y'
WHERE
( 
    SELECT 
    COUNT(*) 
    FROM 
    swc.station_daily_temperatures b 
    WHERE 
        b.stationid = a.stationid 
    AND b.gldasnear_tmin IS NOT NULL
) > 0;

UPDATE swc.station_stats a
SET 
has_some_ncei = 'N'
WHERE
a.has_some_ncei IS NULL;

UPDATE swc.station_stats a
SET 
has_complete_ncei = 'N'
WHERE
a.has_complete_ncei IS NULL;

UPDATE swc.station_stats a
SET 
has_prism = 'N'
WHERE
a.has_prism IS NULL;

UPDATE swc.station_stats a
SET 
has_nldas = 'N'
WHERE
a.has_nldas IS NULL;

UPDATE swc.station_stats a
SET 
has_gldas = 'N'
WHERE
a.has_gldas IS NULL;

UPDATE swc.station_stats a
SET 
has_gldasnear = 'N'
WHERE
a.has_gldasnear IS NULL;
```

#### As the swc user in the database swcdb

Logic Flow:
* If have PRISM data, then use it
* Else use NLDAS data
* Else use GLDAS data
* Else use GLDAS Near data

```
UPDATE swc.station_stats a
SET 
source_used = CASE
WHEN a.has_prism = 'Y'
THEN
   'PRISM'
WHEN a.has_prism = 'N' AND a.has_nldas = 'Y'
THEN
   'NLDAS'
WHEN a.has_prism = 'N' AND a.has_nldas = 'N' AND a.has_gldas = 'Y'
THEN
   'GLDAS'
WHEN a.has_prism = 'N' AND a.has_nldas = 'N' AND a.has_gldas = 'N' AND a.has_gldasnear = 'Y'
THEN
   'GLDASNEAR'
ELSE
   'NODATA'
END;
```

Stats:
 * PRISM: 4681
 * NLDAS: 0
 * GLDAS: 328
 * GLDAS NEAR: 90
 * NO DATA: 45

```
DO $$DECLARE
   rec RECORD;
BEGIN

    FOR rec IN (
        SELECT
         a.stationid
        ,a.source_used
        FROM
        swc.station_stats a
        WHERE
        a.source_used != 'NODATA'
        ORDER BY 
		a.stationid 
    )
    LOOP
    
        EXECUTE '
        COPY (
            SELECT
             b.stationid
            ,TO_CHAR(b.station_date,''YYYY'') AS "year"
            ,TO_CHAR(b.station_date,''MM'')   AS "month"
            ,TO_CHAR(b.station_date,''DD'')   AS "day"
            ,CASE
             WHEN ''' || rec.source_used || ''' = ''PRISM''
             THEN
                b.prism_tmax
             WHEN ''' || rec.source_used || ''' = ''NLDAS''
             THEN
                b.nldas_tmax
             WHEN ''' || rec.source_used || ''' = ''GLDAS''
             THEN
                b.gldas_tmax
             WHEN ''' || rec.source_used || ''' = ''GLDASNEAR''
             THEN
                b.gldasnear_tmax
             END AS tmax
            ,CASE
             WHEN ''' || rec.source_used || ''' = ''PRISM''
             THEN
                b.prism_tmin
             WHEN ''' || rec.source_used || ''' = ''NLDAS''
             THEN
                b.nldas_tmin
             WHEN ''' || rec.source_used || ''' = ''GLDAS''
             THEN
                b.gldas_tmin
             WHEN ''' || rec.source_used || ''' = ''GLDASNEAR''
             THEN
                b.gldasnear_tmin
             END AS tmin
            FROM
            swc.station_daily_temperatures b
            WHERE
            b.stationid = ''' || rec.stationid || '''
            ORDER BY
            b.station_date
        )
        TO ''/myfilelocation/' || rec.stationid || '.txt'' 
        WITH DELIMITER E''\t''';
    
    END LOOP;

END$$;
```
